In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import style
import random 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install spacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from tensorflow.keras.optimizers import Adam
from textblob import TextBlob
from textblob import Word

import numpy as np
from numpy import array
import pandas as pd
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
train=pd.read_csv("/content/drive/MyDrive/Disaster_tweets/train.csv")
test=pd.read_csv("/content/drive/MyDrive/Disaster_tweets/test.csv")

In [ ]:
train = train.drop(['keyword', 'location',], axis=1)
test = test.drop(['keyword', 'location',], axis=1)

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

In [ ]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [ ]:
train['text'] = train['text'].apply(lambda x: cont_to_exp(x))
test['text']= test['text'].apply(lambda x: cont_to_exp(x))

In [ ]:
train['text']=train['text'].str.replace('[^\w\s]',"")
test['text']=test['text'].str.replace('[^\w\s]',"")


In [ ]:
train['text']=train['text'].str.replace('[^a-zA-Z#]'," ")
test['text']=test['text'].str.replace('[^a-zA-Z#]'," ")


In [ ]:
import re
train['text'] = train['text'].apply(lambda x: re.sub(r'(http|ftp|https|httpt)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))
test['text'] = test['text'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))

In [ ]:
train['text']=train['text'].apply(lambda x : " ".join ([x for x in x.split() if len(x)>3]))
test['text']=test['text'].apply(lambda x : " ".join ([x for x in x.split() if len(x)>3]))


In [ ]:
from nltk.corpus import stopwords

stop=stopwords.words('english')
train['text']=train['text'].apply(lambda x : " ".join (x for x in x.split() if x not in stop))
test['text']=test['text'].apply(lambda x : " ".join (x for x in x.split() if x not in stop))


In [ ]:
train['text']=train['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
test['text']=test['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
from textblob import Word

train['text']=train['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
test['text']=test['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer= CountVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')

bow= bow_vectorizer.fit_transform(train['text'])
bow_test= bow_vectorizer.fit_transform(test['text'])

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout


In [ ]:
voc_size=5000

In [ ]:
X_train=train
y_train=train['target']

X_test=test

In [ ]:
X_train.shape

(7613, 3)

In [ ]:
messages=X_train.copy()
messages_test=X_test.copy()


In [ ]:
messages['text'][1]

'forest fire near ronge sask canada'

In [ ]:
messages_test['text'][1]

'heard earthquake different city stay safe everyone'

In [ ]:
messages.reset_index(inplace=True)
messages_test.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus1 = []
for i in range(0, len(messages_test)):
    print(i)
    review1 = re.sub('[^a-zA-Z]', ' ', messages_test['text'][i])
    review1 = review1.lower()
    review1 = review1.split()
    
    review1 = [ps.stem(word) for word in review1 if not word in stopwords.words('english')]
    review1 = ' '.join(review1)
    corpus1.append(review1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2363, 720, 4276, 2299, 2137],
 [1559, 4877, 2317, 2097, 546, 1558],
 [111, 880, 2744, 4954, 4996, 744, 4977, 2744, 4954, 2733, 3043],
 [2641, 2704, 4006, 4977, 2733, 1333],
 [1418, 2027, 1191, 2815, 1169, 4006, 4754, 4699],
 [509, 4703, 1333, 23, 1475, 2543, 3777, 4877, 4135, 4006],
 [4296, 1298, 3044, 2688, 3841, 438, 4296, 923, 4872, 1291, 258, 15],
 [479, 4877, 1932],
 [2650, 4977, 3184, 4631, 2614, 923],
 [1570, 82, 2319, 15],
 [776, 2641, 242, 595, 389],
 [2992,
  3762,
  2728,
  4863,
  4296,
  3192,
  728,
  1651,
  3762,
  2728,
  2822,
  2822,
  3479,
  4296],
 [2688, 4296, 4958, 2860, 2728, 477, 1897, 810],
 [4296, 932, 4197, 67, 932],
 [495, 4699, 2967, 1865, 3270],
 [1497],
 [3073, 4194],
 [1734, 3073],
 [4874],
 [702],
 [1749],
 [1320, 2956],
 [3073, 3758],
 [4877],
 [24],
 [2431, 518, 4518],
 [4759, 767],
 [3073, 4416],
 [2086],
 [1278, 3255],
 [],
 [3054, 1055, 384, 1740, 3178],
 [690, 1724, 3044, 3463, 216, 566],
 [2433, 3270, 3172, 4584, 1740, 3234, 3518],
 [4715, 17

In [ ]:
onehot_repr1=[one_hot(words,voc_size)for words in corpus1]
onehot_repr1

[[3184, 3399, 1865],
 [703, 4276, 2009, 2771, 1835, 980, 3096],
 [1559, 4877, 4906, 2078, 333, 1513, 2614, 923, 518, 252],
 [2486, 4329, 4770, 4006],
 [438, 1984, 4926, 1864, 1803],
 [3125, 4276],
 [645, 4404, 2620, 3034, 72, 3553, 2427],
 [],
 [2664],
 [1944],
 [1278, 2622],
 [1399, 432],
 [3299],
 [],
 [4566],
 [1049, 1055, 384, 1740, 3594, 4877, 3270, 1049, 1055, 384, 4719],
 [2800, 270, 3113, 4185, 1740],
 [3687, 1551, 3634, 3522, 1942, 2361, 1589, 471, 1740, 2686],
 [4064, 264, 3353, 3234, 4189, 2508, 2049, 1333],
 [4742, 2452, 3471, 4883, 1740, 119, 4883, 1740],
 [4494, 2393, 1740, 1989, 4508, 3045],
 [816, 1740, 1580, 1095, 1911, 3869],
 [1780, 2846, 2641, 3770, 1490, 4529, 4480, 2331, 1740, 1977],
 [3455, 1256, 828, 2361, 1740, 4112, 3163, 242, 816, 3578, 1740, 870, 1869],
 [2361, 1740, 3234],
 [2278, 1649, 2165, 4011, 2080, 1740, 299, 3486, 2706, 636, 2274, 4877],
 [1314, 2872, 1740, 4506, 1011, 4791, 3963, 179],
 [816,
  2754,
  1450,
  950,
  2771,
  2386,
  4631,
  3233,
  

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4276 2299 2137]
 [   0    0    0 ... 2097  546 1558]
 [   0    0    0 ... 4954 2733 3043]
 ...
 [   0    0    0 ...  973 3484  522]
 [   0    0    0 ...   54 4773 3578]
 [   0    0    0 ... 3594 3753 3307]]


In [ ]:
sent_length=20
embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=sent_length)
print(embedded_docs1)

[[   0    0    0 ... 3184 3399 1865]
 [   0    0    0 ... 1835  980 3096]
 [   0    0    0 ...  923  518  252]
 ...
 [   0    0    0 ... 4803 2291 4491]
 [   0    0    0 ... 4541 3234 1059]
 [   0    0    0 ... 2650 2894 4697]]


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
#from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score

In [ ]:
X_train=train
y_train=train['target']

X_test=test

In [ ]:
len(embedded_docs),y_train.shape

(7613, (7613,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y_train)

X_final_test=np.array(embedded_docs1)

In [ ]:

X_final.shape,y_final.shape

((7613, 20), (7613,))

In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)

ValueError: ignored

In [ ]:
model.fit(X_final,y_final,epochs=5,batch_size=12)

Epoch 1/5
635/635 [==============================] - 151s 218ms/step - loss: 0.5944 - accuracy: 0.6849 - f1_m: 0.3966 - precision_m: 0.4897 - recall_m: 0.3739
Epoch 2/5
635/635 [==============================] - 139s 219ms/step - loss: 0.3711 - accuracy: 0.8518 - f1_m: 0.7908 - precision_m: 0.8818 - recall_m: 0.7542
Epoch 3/5
635/635 [==============================] - 139s 219ms/step - loss: 0.3094 - accuracy: 0.8901 - f1_m: 0.8520 - precision_m: 0.9172 - recall_m: 0.8234
Epoch 4/5
635/635 [==============================] - 140s 221ms/step - loss: 0.2457 - accuracy: 0.9147 - f1_m: 0.8878 - precision_m: 0.9324 - recall_m: 0.8686
Epoch 5/5
635/635 [==============================] - 140s 220ms/step - loss: 0.1795 - accuracy: 0.9386 - f1_m: 0.9135 - precision_m: 0.9493 - recall_m: 0.8992


In [ ]:
ypred=model.predict_classes(X_final_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
ypred=np.argmax(ypred,axis=1)

In [ ]:
ypred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
submission1=pd.DataFrame({"id": test['id'],"target":ypred})

In [ ]:
submission1

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [ ]:
submission1.to_csv('Submission1.csv',index=False)